----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - How to send parallel requests and receive results as events
This notebook demonstrates how to use Definition objects of the Content layer to send parallel requests and receive results as separated events. Historical Pricing will be used here as an example, but this technique applies to all Definition objects of the Content layer.   

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [55]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
from refinitiv.data.content.historical_pricing import Intervals
import asyncio
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f88a45b11c0 {name='workspace'}>

## Retrieve data

### Define a callback to receive events and display responses

In [3]:
def display_response(response, definition, session):
    current_time = datetime.datetime.now().time()
    display('{} response received at {}'.format(response.closure, current_time))
    if response.is_success:
        display(response.data.df)
    else:
        print(response.http_status)    

### Use get_data_async to send requests in parallel and use asyncio to create separated tasks 
Once done, just wait for reponses to be received and displayed...

In [4]:
asyncio.get_event_loop().create_task(
    historical_pricing.events.Definition(
        universe = 'LSEG.L'
    ).get_data_async(
        on_response = display_response, 
        closure = 'LSEG.L Historical Events'
    )
)

asyncio.get_event_loop().create_task(
    historical_pricing.summaries.Definition(
        universe = 'LSEG.L', 
        interval=Intervals.FIVE_MINUTES
    ).get_data_async(
        on_response = display_response,
        closure = 'LSEG.L 5 minutes Historical Summaries'
    )
)

asyncio.get_event_loop().create_task(
    historical_pricing.summaries.Definition(
        universe = 'LSEG.L',
        interval = Intervals.DAILY
    ).get_data_async(
        on_response = display_response,
        closure = 'LSEG.L Daily Historical Summaries'
    )
)

<Task pending name='Task-5' coro=<DataProviderLayer.get_data_async() running at /Users/olivierdavant-refinitiv/opt/anaconda3/envs/rdl/lib/python3.9/site-packages/refinitiv/data/delivery/_data/_data_provider_layer.py:113>>

"['LSEG.L Daily Historical Summaries'] response received at 15:31:36.295303"

LSEG.L,TRDPRC_1,MKT_HIGH,MKT_LOW,ACVOL_UNS,MKT_OPEN,BID,ASK,TRNOVR_UNS,VWAP,MID_PRICE,...,CLS_AUCVOL,OPN_AUCVOL,OPN_AUC,CLS_AUC,INT_AUC,INT_AUCVOL,EX_VOL_UNS,ALL_C_MOVE,ELG_NUMMOV,NAVALUE
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-14,8101.504,8269,8022,818552,8208,8022,8024,6668909108.56,8147.2026,8023,...,191720,8505,8208,8022,<NA>,<NA>,867074,3907,3526,<NA>
2022-11-15,7980.0,8062,7960,354336,8030,7984,7986,2834718747.55,8000.0745,7985,...,142909,6319,8030,7986,<NA>,<NA>,374395,4694,4399,<NA>
2022-11-16,7996.0,8060,7942,383289,8022,7996,7998,3063037947.99,7991.45801,7997,...,168417,4867,8022,7996,<NA>,<NA>,718489,4981,4689,<NA>
2022-11-17,8036.0,8074,7958,250752,8020,8034,8036,2012039935.09,8024.50862,8035,...,119501,7131,8020,8036,<NA>,<NA>,299563,2964,2708,<NA>
2022-11-18,8122.0,8156,8030,449147,8066,8120,8122,3643058057.44674,8111.0593,8121,...,165293,5102,8066,8122,8104,95291,463214,3015,2883,<NA>
2022-11-21,8126.694,8166,8044,426866,8086,8156,8166,3461767094.34,8109.7226,8161,...,107361,8880,8086,8166,<NA>,<NA>,446777,2457,2284,<NA>
2022-11-22,8156.53,8200,8128,276294,8170,8164,8168,2255065047.70422,8161.8288,8166,...,126394,12168,8170,8164,<NA>,<NA>,302588,3512,3218,<NA>
2022-11-23,8222.2937,8264,8140,230793,8166,8232,8234,1898970901.9867,8228.0373,8233,...,129204,3255,8166,8232,<NA>,<NA>,258016,2420,2256,<NA>
2022-11-24,8110.2397,8250,8054,284774,8242,8080,8082,2308312978.82,8105.7607,8081,...,125594,4881,8242,8080,<NA>,<NA>,342624,3255,2998,<NA>


"['LSEG.L 5 minutes Historical Summaries'] response received at 15:31:36.533460"

LSEG.L,HIGH_1,LOW_1,OPEN_PRC,TRDPRC_1,NUM_MOVES,ACVOL_UNS,HIGH_YLD,LOW_YLD,OPEN_YLD,YIELD,...,BID_NUMMOV,ASK_HIGH_1,ASK_LOW_1,OPEN_ASK,ASK,ASK_NUMMOV,MID_HIGH,MID_LOW,MID_OPEN,MID_PRICE
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-12 12:55:00,7563.8,7560.502,7563.5293,7563.8,5,410,<NA>,<NA>,<NA>,<NA>,...,178,7564,7562,7564,7564,178,7563,7560,7562,7562
2022-12-12 13:00:00,7588.0,7562.0,7562.0,7588.0,64,2644,<NA>,<NA>,<NA>,<NA>,...,738,7592,7562,7564,7592,738,7590,7560,7563,7589
2022-12-12 13:05:00,7602.0,7588.0,7590.494,7600.0,92,4359,<NA>,<NA>,<NA>,<NA>,...,957,7604,7588,7592,7600,957,7602,7586,7589,7598
2022-12-12 13:10:00,7596.0,7572.0,7596.0,7584.0,94,4693,<NA>,<NA>,<NA>,<NA>,...,897,7600,7574,7600,7586,897,7598,7572,7598,7584
2022-12-12 13:15:00,7588.0302,7582.0,7584.998,7582.0,24,1257,<NA>,<NA>,<NA>,<NA>,...,224,7592,7582,7586,7584,224,7589,7581,7584,7582
2022-12-12 13:20:00,7580.0,7560.0,7580.0,7560.0,12,322,<NA>,<NA>,<NA>,<NA>,...,424,7584,7562,7584,7564,424,7582,7560,7582,7562
2022-12-12 13:25:00,7563.0,7556.0,7560.0,7563.0,33,1198,<NA>,<NA>,<NA>,<NA>,...,625,7566,7556,7564,7564,625,7564,7553,7562,7561
2022-12-12 13:30:00,7564.0,7552.0,7552.0,7558.0,21,978,<NA>,<NA>,<NA>,<NA>,...,431,7568,7552,7564,7560,431,7566,7550,7561,7558
2022-12-12 13:35:00,7558.0,7558.0,7558.0,7558.0,2,93,<NA>,<NA>,<NA>,<NA>,...,173,7560,7558,7560,7560,173,7558,7555,7558,7558


"['LSEG.L Historical Events'] response received at 15:31:36.854439"

LSEG.L,EVENT_TYPE,RTL,SEQNUM,TRDXID_1,TRDPRC_1,TRDVOL_1,VWAP,BID,BIDSIZE,ASK,...,MMT_CLASS,TR_TRD_FLG,ACVOL_UNS,OPEN_PRC,HIGH_1,LOW_1,MID_PRICE,IMB_SH,IMB_SIDE,QUALIFIERS
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-12 14:31:31.736,quote,47104,4078705,<NA>,<NA>,<NA>,<NA>,7558,198,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[ASK_TONE]
2022-12-12 14:31:31.736,quote,47136,4078708,<NA>,<NA>,<NA>,<NA>,7558,323,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[BID_TONE]
2022-12-12 14:31:31.736,quote,47152,4078709,<NA>,<NA>,<NA>,<NA>,7558,323,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[ASK_TONE]
2022-12-12 14:31:31.736,quote,47168,4078710,<NA>,<NA>,<NA>,<NA>,7558,323,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[ASK_TONE]
2022-12-12 14:31:31.736,quote,47120,4078707,<NA>,<NA>,<NA>,<NA>,7558,290,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[BID_TONE]
2022-12-12 14:31:31.737,quote,47184,4078712,<NA>,<NA>,<NA>,<NA>,7558,323,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[ASK_TONE]
2022-12-12 14:31:31.737,quote,47200,4078713,<NA>,<NA>,<NA>,<NA>,7558,323,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[ASK_TONE]
2022-12-12 14:31:31.737,quote,47216,4078714,<NA>,<NA>,<NA>,<NA>,7558,449,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[BID_TONE]
2022-12-12 14:31:31.745,quote,47232,4078717,<NA>,<NA>,<NA>,<NA>,7558,416,7564,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7561,<NA>,<NA>,[BID_TONE]


## Close the session

In [5]:
rd.close_session()